增量赋值运算符 += 和 *= 的表现取决于它们的第一个操作对象。简单起见，我们把讨论集中在增量加法（+=）上，但是这些概念对 *= 和其他增量运算符来说都是一样的。
+= 背后的特殊方法是 __iadd__（用于“就地加法”）。但是如果一个类没有实现这个方法的话，Python 会退一步调用 __add__。考虑下面这个简单的表达式：
    >>> a += b
如 果 a 实现了 __iadd__ 方 法， 就 会 调 用 这 个 方 法。 同 时 对 可 变 序 列（ 例 如 list、bytearray 和 array.array）来说，a 会就地改动，就像调用了 a.extend(b) 一样。
但是如果 a 没有实现 __iadd__ 的话，a += b 这个表达式的效果就变得跟 a = a + b 一样了：
首先计算 a + b，得到一个新的对象，然后赋值给 a。也就是说，在这个表达式中，变量名会不会被关联到新的对象，完全取决于这个类型有没有实现 __iadd__ 这个方法。
总体来讲，可变序列一般都实现了 __iadd__ 方法，因此 += 是就地加法。而不可变序列根本就不支持这个操作，对这个方法的实现也就无从谈起。
接下来有个小例子，展示的是 *= 在可变和不可变序列上的作用

In [1]:
l = [1, 2, 3]
id(l)

2057341785792

In [2]:
l *= 2
l

[1, 2, 3, 1, 2, 3]

In [3]:
id(l)

2057341785792

In [4]:
t = (1, 2, 3)
id(t)

2057336491968

In [5]:
t *= 2
t

(1, 2, 3, 1, 2, 3)

In [6]:
id(t)

2057336908384

对不可变序列进行重复拼接操作的话，效率会很低，因为每次都有一个新对象，而解释器需要把原来对象中的元素先复制到新的对象里，然后再追加新的元素。
我们已经认识了 += 的一般用法，下面来看一个有意思的边界情况。这个例子可以说是突出展示了“不可变性”对于元组来说到底意味着什么。
### 一个关于+=的谜题
读完下面的代码，然后回答这个问题：示例 2-14 中的两个表达式到底会产生什么结果？回答之前不要用控制台去运行这两个式子。
#### 示例 2-14 一个谜题

In [7]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

到底会发生下面 4 种情况中的哪一种？
a. t 变成 (1, 2, [30, 40, 50, 60])。
b. 因为 tuple 不支持对它的元素赋值，所以会抛出 TypeError 异常。
c. 以上两个都不是。
d. a 和 b 都是对的。
答案其实是d，也是就是a和b都是对的！示例 2-15 是运行这段代码得到的结果
#### 示例 2-15 没人料到的结果：t[2] 被改动了，但是也有异常抛出

In [8]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [9]:
t

(1, 2, [30, 40, 50, 60])

至此我得到了 3 个教训。
• 不要把可变对象放在元组里面。
• 增量赋值不是一个原子操作。我们刚才也看到了，它虽然抛出了异常，但还是完成了操作。
• 查看 Python 的字节码并不难，而且它对我们了解代码背后的运行机制很有帮助。